In [1]:
import pyspark


sc = pyspark.SparkContext('local[*]')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/02 09:32:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

[624, 365, 670, 687, 328]

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


rdd = sc.parallelize([
    {"Name": "Vasia", "Salary": 1000, "Gender": "Male"},
    {"Name": "Peria", "Salary": 500, "Gender": "Male"},
    {"Name": "Sveta", "Salary": 200, "Gender": "Famale"}
])

spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(rdd)
df.show()
df

+------+-----+------+
|Gender| Name|Salary|
+------+-----+------+
|  Male|Vasia|  1000|
|  Male|Peria|   500|
|Famale|Sveta|   200|
+------+-----+------+



DataFrame[Gender: string, Name: string, Salary: bigint]

In [26]:
df.select("Salary").show()

+------+
|Salary|
+------+
|  1000|
|   500|
|   200|
+------+



In [27]:
df.filter(col("Name") == "Vasia").show()

+------+-----+------+
|Gender| Name|Salary|
+------+-----+------+
|  Male|Vasia|  1000|
+------+-----+------+



In [29]:
df.filter("Name == 'Vasia' or Name = 'Sveta'").select("Salary").show()

+------+
|Salary|
+------+
|  1000|
|   200|
+------+



In [30]:
orgs_df = spark.read.option("multiline","true").json("github.json")
orgs_df.printSchema()

root
 |-- avatar_url: string (nullable = true)
 |-- description: string (nullable = true)
 |-- events_url: string (nullable = true)
 |-- hooks_url: string (nullable = true)
 |-- id: string (nullable = true)
 |-- issues_url: string (nullable = true)
 |-- login: string (nullable = true)
 |-- members_url: string (nullable = true)
 |-- node_id: string (nullable = true)
 |-- public_members_url: string (nullable = true)
 |-- repos_url: string (nullable = true)
 |-- url: string (nullable = true)



In [31]:
orgs_df.show()

+--------------------+--------------------+--------------------+--------------------+----+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          avatar_url|         description|          events_url|           hooks_url|  id|          issues_url|            login|         members_url|             node_id|  public_members_url|           repos_url|                 url|
+--------------------+--------------------+--------------------+--------------------+----+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|https://avatars.g...|                    |https://api.githu...|https://api.githu...|  44|https://api.githu...|          errfree|https://api.githu...|MDEyOk9yZ2FuaXphd...|https://api.githu...|https://api.githu...|https://api.githu...|
|https://avatars.g...|                    |https://api.githu

In [32]:
orgs_df.select("repos_url").show()

+--------------------+
|           repos_url|
+--------------------+
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
|https://api.githu...|
+--------------------+
only showing top 20 rows



In [33]:
orgs_df.groupBy("description").count().show()

+--------------------+-----+
|         description|count|
+--------------------+-----+
|                   7|    1|
|                   3|    2|
|We build software...|    1|
|                   5|    2|
|                   6|    1|
|Leading Spree Com...|    1|
|We're an agile te...|    1|
|                   1|    3|
|                   4|    2|
|                    |   12|
|                  4.|    1|
|                   2|    3|
+--------------------+-----+



In [42]:
from pyspark.sql.functions import udf, lit
from pyspark.sql.types import StringType, FloatType


@udf(returnType=StringType()) 
def uppercase(value: str):
    return value.upper()


@udf(returnType=StringType())
def inversion(value: str):
    return value[::-1]


@udf(returnType=FloatType())
def calculation_of_taxes(value: float, percent: float):
    return value - (value * percent) / 100 
    

df.withColumn("Upper", uppercase(col("Name"))).show()
df.withColumn("Inversion Name", inversion(col("Name"))).show()
df.withColumn("Salary after taxes", calculation_of_taxes(col("Salary"), lit(13))).show()

+------+-----+------+-----+
|Gender| Name|Salary|Upper|
+------+-----+------+-----+
|  Male|Vasia|  1000|VASIA|
|  Male|Peria|   500|PERIA|
|Famale|Sveta|   200|SVETA|
+------+-----+------+-----+

+------+-----+------+--------------+
|Gender| Name|Salary|Inversion Name|
+------+-----+------+--------------+
|  Male|Vasia|  1000|         aisaV|
|  Male|Peria|   500|         aireP|
|Famale|Sveta|   200|         atevS|
+------+-----+------+--------------+

+------+-----+------+------------------+
|Gender| Name|Salary|Salary after taxes|
+------+-----+------+------------------+
|  Male|Vasia|  1000|             870.0|
|  Male|Peria|   500|             435.0|
|Famale|Sveta|   200|             174.0|
+------+-----+------+------------------+

